## EV Adoption & Charging Infrastructure Analysis

This notebook contains the full data preparation and integration workflow used to build the final visualisation for the project. It includes cleaning three datasets, merging them into a unified analytical table, and preparing the processed data for the explanatory animated bubble scatterplot.


# Electric Vehicle Population - Data Cleaning (Washington State DMV)

This section loads the raw DMV electric vehicle population dataset, cleans missing values, removes ID-only columns, and saves a cleaned version for further integration.

## Step 1: Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 2: Load the Dataset

In [ ]:
ev_data = pd.read_csv('/content/drive/MyDrive/Projects - HA/DMV/electric_vehicle_population_data.csv')

In [ ]:
print (ev_data)

        VIN (1-10)     County       City State  Postal Code  Model Year  \
0       WA1E2AFY8R   Thurston    Olympia    WA      98512.0        2024   
1       WAUUPBFF4J     Yakima     Wapato    WA      98951.0        2018   
2       1N4AZ0CP0F       King    Seattle    WA      98125.0        2015   
3       WA1VAAGE5K       King       Kent    WA      98031.0        2019   
4       7SAXCAE57N  Snohomish    Bothell    WA      98021.0        2022   
...            ...        ...        ...   ...          ...         ...   
264623  KM8KRDDF7R       King  Sammamish    WA      98074.0        2024   
264624  1N4BZ0CPXG       King    Seattle    WA      98199.0        2016   
264625  7SAYGDEE4P       King     Renton    WA      98059.0        2023   
264626  5YJYGDEE9M       King    Redmond    WA      98052.0        2021   
264627  7SAYGDEE9S       King   Bellevue    WA      98004.0        2025   

           Make    Model                   Electric Vehicle Type  \
0          AUDI     Q5 E  Plug-

## Step 3: Basic structure and summary

In this step I examine the overall structure of the dataset.


In [ ]:
print ("Rows, Columns:", ev_data.shape)

Rows, Columns: (264628, 17)


In [ ]:
ev_data.shape

(264628, 17)

In [ ]:
ev_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264628 entries, 0 to 264627
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         264628 non-null  object 
 1   County                                             264619 non-null  object 
 2   City                                               264619 non-null  object 
 3   State                                              264628 non-null  object 
 4   Postal Code                                        264619 non-null  float64
 5   Model Year                                         264628 non-null  int64  
 6   Make                                               264628 non-null  object 
 7   Model                                              264628 non-null  object 
 8   Electric Vehicle Type                              264628 non-null  object

In [ ]:
ev_data.describe()

,Postal Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,2020 Census Tract
count,264619.000000,264628.000000,264624.000000,264624.000000,263969.000000,2.646280e+05,2.646190e+05
mean,98170.770500,2021.842556,41.713159,678.902197,28.862897,2.422540e+08,5.297109e+10
std,2637.717847,3.044095,80.377977,6868.919926,14.883356,6.516028e+07,1.638317e+09
min,1469.000000,1999.000000,0.000000,0.000000,1.000000,4.385000e+03,1.001020e+09
25%,98052.000000,2021.000000,0.000000,0.000000,17.000000,2.174474e+08,5.303301e+10
50%,98133.000000,2023.000000,0.000000,0.000000,32.000000,2.603598e+08,5.303303e+10
75%,98382.000000,2024.000000,34.000000,0.000000,42.000000,2.758921e+08,5.305307e+10
max,99577.000000,2026.000000,337.000000,845000.000000,49.000000,4.791150e+08,6.601095e+10


## Step 4: Check missing values

To systematically inspect missing data.

In [ ]:
missing_values_new = ev_data.isnull().sum().sort_values(ascending=False)
missing_values_new

,0
Legislative District,659
Vehicle Location,17
2020 Census Tract,9
County,9
City,9
Postal Code,9
Electric Utility,9
Base MSRP,4
Electric Range,4
VIN (1-10),0



## Step 5: Drop unnecessary identifier columns

These fields do not add analytical value for the visualisation assignment and may introduce noise or privacy concerns. Therefore, I drop them using:


In [ ]:
cols_to_drop = [
    'VIN (1-10)',
    'DOL Vehicle ID',
    'Legislative District'
]

ev_drop = ev_data.drop(columns=cols_to_drop)
ev_drop.shape


(264628, 14)


## Step 6: Handle missing values

After dropping the unnecessary columns, I focus on handling the remaining missing values in `ev_drop`. The goal is to use **sensible, data-driven imputation** methods that preserve the structure of the dataset.

In [ ]:
# 1. Fill numeric missing values
ev_drop['Electric Range'] = ev_drop['Electric Range'].fillna(ev_drop['Electric Range'].median())
ev_drop['Base MSRP'] = ev_drop['Base MSRP'].fillna(ev_drop['Base MSRP'].median())

# 2. Fill postal code missing values
ev_drop['Postal Code'] = ev_drop['Postal Code'].fillna(0)
ev_drop['Postal Code'] = ev_drop['Postal Code'].astype(int)

# 3. Fill categorical missing values
ev_drop['County'] = ev_drop['County'].fillna(ev_drop['County'].mode()[0])
ev_drop['City'] = ev_drop['City'].fillna(ev_drop['City'].mode()[0])
ev_drop['Electric Utility'] = ev_drop['Electric Utility'].fillna(ev_drop['Electric Utility'].mode()[0])

# 4. Fill missing vehicle location
ev_drop['Vehicle Location'] = ev_drop['Vehicle Location'].fillna("Unknown")

# 5. Drop unnecessary column if it exists
if '2020 Census Tract' in ev_drop.columns:
    ev_drop = ev_drop.drop(columns=['2020 Census Tract'])

# 6. Final check
ev_drop.isnull().sum()



,0
County,0
City,0
State,0
Postal Code,0
Model Year,0
Make,0
Model,0
Electric Vehicle Type,0
Clean Alternative Fuel Vehicle (CAFV) Eligibility,0
Electric Range,0



## Step 7: Save the cleaned dataset

To reuse the cleaned data and submit it as part of the assignment, I save the `ev_drop` DataFrame as a new CSV file:



In [ ]:
# Save cleaned dataset as CSV
ev_drop.to_csv('/content/drive/MyDrive/Projects - HA/DMV/cleaned_electric_vehicle_population_data.csv', index=False)

## How to check if the file is saved properly

In [ ]:
import os
os.path.exists('/content/drive/MyDrive/Projects - HA/DMV/cleaned_electric_vehicle_population_data.csv')

True

## Step 1: Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np

## Step 2: Load the Dataset ( Alternative Fuel Stations )

This dataset contains information about alternative fuel stations across the United States, including EV charging stations. It includes station attributes, location coordinates, network operators, and operational status.

In [ ]:
stations = pd.read_csv('/content/drive/MyDrive/Projects - HA/DMV/alt_fuel_stations (Feb 18 2017).csv')

stations.head()


,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,Updated At,Owner Type Code,Federal Agency ID,Federal Agency Name,Open Date,Hydrogen Status Link,NG Vehicle Class,LPG Primary,E85 Blender Pump,EV Connector Types
0,CNG,Alagasco Montgomery Operations Center,2951 Chestnut St,NaN,Montgomery,AL,36107.0,NaN,NaN,E,...,2017-01-18 02:53:29 UTC,T,NaN,NaN,2010-12-01,NaN,MD,NaN,NaN,NaN
1,CNG,Clean Energy - PS Energy - Atlanta,340 Whitehall St,"From I-7585 N, exit 91 to Central Avenue, left...",Atlanta,GA,30303.0,NaN,866-809-4869,E,...,2017-01-18 02:51:45 UTC,P,NaN,NaN,1994-07-15,NaN,MD,NaN,NaN,NaN
2,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NaN,Atlanta,GA,30324.0,NaN,NaN,E,...,2017-01-18 01:19:51 UTC,LG,NaN,NaN,1996-12-15,NaN,LD,NaN,NaN,NaN
3,CNG,United Parcel Service,270 Marvin Miller Dr,NaN,Atlanta,GA,30336.0,NaN,NaN,E,...,2017-01-18 01:58:34 UTC,P,NaN,NaN,1997-01-01,NaN,HD,NaN,NaN,NaN
4,CNG,Clean Energy - Texas Department of Transportation,7721A Washington St,"I-10, Washington Ave exit, 1.5 blocks to the s...",Houston,TX,77007.0,NaN,866-809-4869,E,...,2017-01-18 01:48:57 UTC,P,NaN,NaN,1996-12-15,NaN,MD,NaN,NaN,NaN


## Step 3: Inspect Structure and Summary

First, I examine the number of rows, columns, data types, and completeness of the dataset.

In [ ]:
stations.shape
stations.info()
stations.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27922 entries, 0 to 27921
Data columns (total 38 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Fuel Type Code           27922 non-null  object 
 1   Station Name             27922 non-null  object 
 2   Street Address           27904 non-null  object 
 3   Intersection Directions  10999 non-null  object 
 4   City                     27921 non-null  object 
 5   State                    27920 non-null  object 
 6   ZIP                      27921 non-null  float64
 7   Plus4                    0 non-null      float64
 8   Station Phone            25093 non-null  object 
 9   Status Code              27922 non-null  object 
 10  Expected Date            230 non-null    object 
 11  Groups With Access Code  27922 non-null  object 
 12  Access Days Time         24435 non-null  object 
 13  Cards Accepted           7690 non-null   object 
 14  BD Blends             

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,Updated At,Owner Type Code,Federal Agency ID,Federal Agency Name,Open Date,Hydrogen Status Link,NG Vehicle Class,LPG Primary,E85 Blender Pump,EV Connector Types
count,27922,27922,27904,10999,27921,27920,27921.000000,0.0,25093,27922,...,27922,18119,600.000000,600,14515,38,1981,3664,3176,18216
unique,7,18806,25046,10129,5224,53,NaN,NaN,11873,3,...,16992,6,NaN,19,1874,38,3,2,2,47
top,ELEC,U-Haul,One American Road,STATION 01,Los Angeles,CA,NaN,NaN,888-758-4389,E,...,2017-01-18 00:27:43 UTC,P,NaN,Department of Air Force,2012-01-31,http://cafcp.org/stations/thousand-palms-sunline,HD,False,False,J1772
freq,18266,1023,26,149,405,5236,NaN,NaN,5809,27610,...,8,14651,NaN,106,802,1,1495,3364,2819,12645
mean,NaN,NaN,NaN,NaN,NaN,NaN,58049.611618,NaN,NaN,NaN,...,NaN,NaN,13.273333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,30466.515158,NaN,NaN,NaN,...,NaN,NaN,6.836899,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,969.000000,NaN,NaN,NaN,...,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,31401.000000,NaN,NaN,NaN,...,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,60025.000000,NaN,NaN,NaN,...,NaN,NaN,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,90034.000000,NaN,NaN,NaN,...,NaN,NaN,19.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Key observations

Dataset contains 27,922 rows and 38 columns.

Many columns have extremely high missing values.

Several fields are administrative or rarely used (e.g., NG PSI, BD Blends).

Location fields (Latitude, Longitude) are complete and very important.

EV-related metrics (EV Level1/Level2/DC Fast Count) are retained for visualisation.

## Step 4: Check Missing Values

Many columns contain more than 80-90% missing values, making them unsuitable for meaningful analysis.

In [ ]:
stations.isnull().sum().sort_values(ascending=False)

,0
Plus4,27922
Hydrogen Status Link,27884
EV Other Info,27810
Expected Date,27692
Federal Agency Name,27322
Federal Agency ID,27322
BD Blends,27192
EV Level1 EVSE Num,26242
NG PSI,26155
NG Fill Type Code,26153


## Step 5: Drop Columns With Extremely High Missing Values

Many columns in this dataset contain more than 80-99% missing values.
These fields do not contribute to EV charging network analysis and would reduce the clarity of visualisations.
Therefore, I drop the non-essential administrative or incomplete fields.

In [ ]:
cols_to_drop = [
    'Plus4', 'Hydrogen Status Link', 'EV Other Info', 'Expected Date',
    'Federal Agency ID', 'Federal Agency Name', 'BD Blends', 'NG Fill Type Code',
    'NG PSI', 'NG Vehicle Class', 'LPG Primary', 'E85 Blender Pump',
    'Cards Accepted', 'Intersection Directions', 'Owner Type Code'
]

stations_clean = stations.drop(columns=cols_to_drop)


## Step 6: Clean ZIP Codes (Convert to Int + Fill NA)

In [ ]:
stations_clean['ZIP'] = stations_clean['ZIP'].fillna(0).astype(int)

## Step 7: Fill Missing Numeric EV Charging Counts

In [ ]:
num_cols = ['EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count']

for col in num_cols:
    stations_clean[col] = stations_clean[col].fillna(0)

## Step 8: Fill Categorical Missing Values

In [ ]:
cat_cols = [
    'Street Address', 'City', 'State', 'Station Phone',
    'Access Days Time', 'EV Network', 'EV Network Web', 'EV Connector Types'
]

for col in cat_cols:
    stations_clean[col] = stations_clean[col].fillna("Unknown")

## Step 9: Handle Date Columns for Velocity Aspect

The dataset includes time-based fields that enable temporal analysis of EV infrastructure growth.
To keep the velocity aspect, I fill or convert these fields.

In [ ]:
# 1. Fill Date Last Confirmed
stations_clean['Date Last Confirmed'] = stations_clean['Date Last Confirmed'].fillna("Unknown")

# 2. Handle 'Open Date' safely
if 'Open Date' in stations_clean.columns:
    # Convert to datetime
    stations_clean['Open Date'] = pd.to_datetime(stations_clean['Open Date'], errors='coerce')

    # Extract year for velocity analysis
    stations_clean['Open Year'] = stations_clean['Open Date'].dt.year.fillna("Unknown")

    # Drop the original date column
    stations_clean = stations_clean.drop(columns=['Open Date'])


## Step 10: Save the Cleaned Dataset

After completing all cleaning steps for the Alternative Fuel Stations dataset, I save the processed DataFrame as a new CSV file.

In [ ]:
# Save the cleaned alternative fuel stations dataset
stations_clean.to_csv('/content/drive/MyDrive/Projects - HA/DMV/cleaned_alt_fuel_stations.csv', index=False)

## Step 1: Load the EV specifications dataset

This dataset contains technical specifications of electric vehicles, such as battery capacity, top speed, range, torque, acceleration, and body type.

In [ ]:
import pandas as pd
import numpy as np

spec = pd.read_csv('/content/drive/MyDrive/Projects - HA/DMV/electric_vehicles_spec_2025.csv')

spec.head()

,brand,model,top_speed_kmh,battery_capacity_kWh,battery_type,number_of_cells,torque_nm,efficiency_wh_per_km,range_km,acceleration_0_100_s,...,towing_capacity_kg,cargo_volume_l,seats,drivetrain,segment,length_mm,width_mm,height_mm,car_body_type,source_url
0,Abarth,500e Convertible,155,37.8,Lithium-ion,192.0,235.0,156,225,7.0,...,0.0,185,4,FWD,B - Compact,3673,1683,1518,Hatchback,https://ev-database.org/car/1904/Abarth-500e-C...
1,Abarth,500e Hatchback,155,37.8,Lithium-ion,192.0,235.0,149,225,7.0,...,0.0,185,4,FWD,B - Compact,3673,1683,1518,Hatchback,https://ev-database.org/car/1903/Abarth-500e-H...
2,Abarth,600e Scorpionissima,200,50.8,Lithium-ion,102.0,345.0,158,280,5.9,...,0.0,360,5,FWD,JB - Compact,4187,1779,1557,SUV,https://ev-database.org/car/3057/Abarth-600e-S...
3,Abarth,600e Turismo,200,50.8,Lithium-ion,102.0,345.0,158,280,6.2,...,0.0,360,5,FWD,JB - Compact,4187,1779,1557,SUV,https://ev-database.org/car/3056/Abarth-600e-T...
4,Aiways,U5,150,60.0,Lithium-ion,NaN,310.0,156,315,7.5,...,NaN,496,5,FWD,JC - Medium,4680,1865,1700,SUV,https://ev-database.org/car/1678/Aiways-U5


## Step 2: Inspect structure and summary

Here I check the size, column types, and basic statistics to understand the structure of the EV specifications dataset.

In [ ]:
spec.shape
spec.info()
spec.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   brand                      478 non-null    object 
 1   model                      477 non-null    object 
 2   top_speed_kmh              478 non-null    int64  
 3   battery_capacity_kWh       478 non-null    float64
 4   battery_type               478 non-null    object 
 5   number_of_cells            276 non-null    float64
 6   torque_nm                  471 non-null    float64
 7   efficiency_wh_per_km       478 non-null    int64  
 8   range_km                   478 non-null    int64  
 9   acceleration_0_100_s       478 non-null    float64
 10  fast_charging_power_kw_dc  477 non-null    float64
 11  fast_charge_port           477 non-null    object 
 12  towing_capacity_kg         452 non-null    float64
 13  cargo_volume_l             477 non-null    object 

,brand,model,top_speed_kmh,battery_capacity_kWh,battery_type,number_of_cells,torque_nm,efficiency_wh_per_km,range_km,acceleration_0_100_s,...,towing_capacity_kg,cargo_volume_l,seats,drivetrain,segment,length_mm,width_mm,height_mm,car_body_type,source_url
count,478,477,478.000000,478.000000,478,276.000000,471.000000,478.000000,478.000000,478.000000,...,452.000000,477,478.000000,478,478,478.000000,478.000000,478.000000,478,478
unique,59,477,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,140,NaN,3,15,NaN,NaN,NaN,8,478
top,Mercedes-Benz,X Privilege AWD (MY25),NaN,NaN,Lithium-ion,NaN,NaN,NaN,NaN,NaN,...,NaN,520,NaN,AWD,JC - Medium,NaN,NaN,NaN,SUV,https://ev-database.org/car/3178/firefly-firefly
freq,42,1,NaN,NaN,478,NaN,NaN,NaN,NaN,NaN,...,NaN,17,NaN,191,91,NaN,NaN,NaN,244,1
mean,NaN,NaN,185.487448,74.043724,NaN,485.293478,498.012739,162.903766,393.179916,6.882636,...,1052.261062,NaN,5.263598,NaN,NaN,4678.506276,1887.359833,1601.125523,NaN,NaN
std,NaN,NaN,34.252773,20.331058,NaN,1210.819733,241.461128,34.317532,103.287335,2.730696,...,737.851774,NaN,1.003961,NaN,NaN,369.210573,73.656807,130.754851,NaN,NaN
min,NaN,NaN,125.000000,21.300000,NaN,72.000000,113.000000,109.000000,135.000000,2.200000,...,0.000000,NaN,2.000000,NaN,NaN,3620.000000,1610.000000,1329.000000,NaN,NaN
25%,NaN,NaN,160.000000,60.000000,NaN,150.000000,305.000000,143.000000,320.000000,4.800000,...,500.000000,NaN,5.000000,NaN,NaN,4440.000000,1849.000000,1514.000000,NaN,NaN
50%,NaN,NaN,180.000000,76.150000,NaN,216.000000,430.000000,155.000000,397.500000,6.600000,...,1000.000000,NaN,5.000000,NaN,NaN,4720.000000,1890.000000,1596.000000,NaN,NaN
75%,NaN,NaN,201.000000,90.600000,NaN,324.000000,679.000000,177.750000,470.000000,8.200000,...,1600.000000,NaN,5.000000,NaN,NaN,4961.000000,1939.000000,1665.000000,NaN,NaN


## Step 3: Check missing values


In [ ]:
spec.isnull().sum().sort_values(ascending=False)

,0
number_of_cells,202
towing_capacity_kg,26
torque_nm,7
fast_charging_power_kw_dc,1
fast_charge_port,1
model,1
cargo_volume_l,1
battery_capacity_kWh,0
top_speed_kmh,0
brand,0


## Step 4: Clean Missing Values (EV Specs Dataset)



In [ ]:
num_cols = ['number_of_cells', 'towing_capacity_kg', 'torque_nm', 'fast_charging_power_kw_dc']

for col in num_cols:
    spec[col] = spec[col].fillna(spec[col].median())

Clean Categorical Text Columns

In [ ]:
spec['model'] = spec['model'].fillna(spec['model'].mode()[0])
spec['fast_charge_port'] = spec['fast_charge_port'].fillna("Unknown")
spec['cargo_volume_l'] = spec['cargo_volume_l'].fillna("Unknown")

## Step 5: Save the Cleaned Dataset



In [ ]:
spec.to_csv('/content/drive/MyDrive/Projects - HA/DMV/cleaned_ev_specs.csv', index=False)

## Step 1: Load all cleaned datasets

In this section, I load the three cleaned datasets that will be integrated:

- **Dataset 1 - EV registrations**  
  `cleaned_electric_vehicle_populationData.csv` (vehicle-level registrations in Washington State)

- **Dataset 2 - Alternative fuel / EV charging stations**  
  `cleaned_alt_fuel_stations.csv` (station-level infrastructure data for the US)

- **Dataset 3 - EV technical specifications**  
  `cleaned_ev_specs.csv` (model-level specs such as range, battery size, acceleration, etc.)

These cleaned files were created in the earlier data cleaning steps.


In [ ]:
import pandas as pd
import numpy as np

# Load cleaned DMV EV registration data (Dataset 1)
ev_reg = pd.read_csv('/content/drive/MyDrive/Projects - HA/DMV/cleaned_electric_vehicle_population_data.csv')

# Load cleaned alternative fuel stations data (Dataset 2)
stations_clean = pd.read_csv('/content/drive/MyDrive/Projects - HA/DMV/cleaned_alt_fuel_stations.csv')

# Load cleaned EV specifications data (Dataset 3)
spec = pd.read_csv('/content/drive/MyDrive/Projects - HA/DMV/cleaned_ev_specs.csv')

# Quick shape checks
print("DMV registrations:", ev_reg.shape)
print("Charging stations:", stations_clean.shape)
print("EV specs:", spec.shape)

DMV registrations: (264628, 13)
Charging stations: (27922, 23)
EV specs: (478, 22)


## Step 2: Prepare vehicle keys for merging DMV registrations with EV specs

The DMV dataset and the EV specs dataset use slightly different naming.To merge these two tables, I create normalized join keys.

In [ ]:
# Create cleaned join keys in DMV data
ev_reg['Make_clean'] = ev_reg['Make'].str.upper().str.strip()
ev_reg['Model_clean'] = ev_reg['Model'].str.upper().str.strip()

# Create cleaned join keys in specs data
spec['brand_clean'] = spec['brand'].str.upper().str.strip()
spec['model_clean'] = spec['model'].str.upper().str.strip()

# Check a few rows to confirm
ev_reg[['Make', 'Model', 'Make_clean', 'Model_clean']].head()
spec[['brand', 'model', 'brand_clean', 'model_clean']].head()

,brand,model,brand_clean,model_clean
0,Abarth,500e Convertible,ABARTH,500E CONVERTIBLE
1,Abarth,500e Hatchback,ABARTH,500E HATCHBACK
2,Abarth,600e Scorpionissima,ABARTH,600E SCORPIONISSIMA
3,Abarth,600e Turismo,ABARTH,600E TURISMO
4,Aiways,U5,AIWAYS,U5


## Step 3: Merge Dataset 1 (EV registrations) with Dataset 3 (EV specifications)

In this step, I enrich the EV registration data with technical vehicle specifications.

I perform a **left join**.


A left join ensures:
- All registered EVs in Washington State are preserved.
- Specs are attached only for models that can be matched by make/brand and model name.


In [ ]:
# Merge DMV registrations (Dataset 1) with EV specs (Dataset 3)
ev_with_specs = pd.merge(
    ev_reg,
    spec,
    left_on=['Make_clean', 'Model_clean'],
    right_on=['brand_clean', 'model_clean'],
    how='left'
)

print("Shape after merging DMV + Specs:", ev_with_specs.shape)

# Drop helper key columns (optional but cleaner)
ev_with_specs = ev_with_specs.drop(columns=['Make_clean', 'Model_clean', 'brand_clean', 'model_clean'])

# Quick check of columns
ev_with_specs.columns

Shape after merging DMV + Specs: (264628, 39)


Index(['County', 'City', 'State', 'Postal Code', 'Model Year', 'Make', 'Model',
       'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Vehicle Location', 'Electric Utility', 'brand', 'model',
       'top_speed_kmh', 'battery_capacity_kWh', 'battery_type',
       'number_of_cells', 'torque_nm', 'efficiency_wh_per_km', 'range_km',
       'acceleration_0_100_s', 'fast_charging_power_kw_dc', 'fast_charge_port',
       'towing_capacity_kg', 'cargo_volume_l', 'seats', 'drivetrain',
       'segment', 'length_mm', 'width_mm', 'height_mm', 'car_body_type',
       'source_url'],
      dtype='object')

## Step 4: Prepare charging stations data for integration

The charging stations dataset contains station-level records for multiple fuel types and multiple US states.

For this project, I only need **electric vehicle infrastructure** that can be linked to the DMV EV registrations in Washington.

In [ ]:
# 1. Filter to electric stations only
stations_ev = stations_clean[stations_clean['Fuel Type Code'] == 'ELEC'].copy()

# 2. Filter to Washington State (WA) to match DMV data
stations_ev_wa = stations_ev[stations_ev['State'] == 'WA'].copy()

print("Electric stations in WA:", stations_ev_wa.shape)

# 3. Aggregate infrastructure per City + State
stations_city = stations_ev_wa.groupby(['City', 'State'], as_index=False).agg({
    'Station Name': 'nunique',          # number of unique stations
    'EV Level1 EVSE Num': 'sum',        # total level 1 ports
    'EV Level2 EVSE Num': 'sum',        # total level 2 ports
    'EV DC Fast Count': 'sum'           # total DC fast chargers
})

# Rename columns to meaningful names
stations_city = stations_city.rename(columns={
    'Station Name': 'num_stations',
    'EV Level1 EVSE Num': 'total_level1_ports',
    'EV Level2 EVSE Num': 'total_level2_ports',
    'EV DC Fast Count': 'total_dc_fast_ports'
})

stations_city.head()

Electric stations in WA: (755, 23)


,City,State,num_stations,total_level1_ports,total_level2_ports,total_dc_fast_ports
0,AUBURN,WA,3,0.0,6.0,6.0
1,Aberdeen,WA,1,0.0,0.0,8.0
2,Algona,WA,1,0.0,2.0,0.0
3,Anacortes,WA,3,0.0,3.0,0.0
4,Arlington,WA,1,0.0,3.0,0.0


## Step 5: Merge EV Registration+Specs dataset with city-level charging infrastructure



In [ ]:
final_merged = pd.merge(
    ev_with_specs,
    stations_city,
    on=['City', 'State'],
    how='left'
)

print("Shape after merging DMV+Specs with Stations:", final_merged.shape)

# Replace missing infrastructure values with 0 for cities with no station records
infra_cols = ['num_stations', 'total_level1_ports', 'total_level2_ports', 'total_dc_fast_ports']

for col in infra_cols:
    final_merged[col] = final_merged[col].fillna(0).astype(int)

# Quick look
final_merged[infra_cols].describe()

Shape after merging DMV+Specs with Stations: (264628, 39)


,num_stations,total_level1_ports,total_level2_ports,total_dc_fast_ports
count,264628.000000,264628.000000,264628.000000,264628.000000
mean,29.483898,12.008574,80.747510,4.631588
std,46.949250,18.843616,135.751626,6.483312
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,4.000000,0.000000
50%,6.000000,2.000000,17.000000,1.000000
75%,28.000000,13.000000,64.000000,6.000000
max,135.000000,52.000000,386.000000,18.000000


## Step 6: Sanity checks on the final merged dataset

To validate the integration steps, I perform a few quick checks.

In [ ]:
# Check row count vs original DMV data
print("Original DMV rows:", ev_reg.shape[0])
print("Final merged rows:", final_merged.shape[0])

# Show a sample of combined info
cols_to_view = [
    'County', 'City', 'State', 'Model Year', 'Make', 'Model',
    'battery_capacity_kWh', 'range_km', 'top_speed_kmh',
    'num_stations', 'total_level2_ports', 'total_dc_fast_ports'
]

final_merged[cols_to_view].head(10)

Original DMV rows: 264628
Final merged rows: 264628


,County,City,State,Model Year,Make,Model,battery_capacity_kWh,range_km,top_speed_kmh,num_stations,total_level2_ports,total_dc_fast_ports
0,Thurston,Olympia,WA,2024,AUDI,Q5 E,NaN,NaN,NaN,30,52,2
1,Yakima,Wapato,WA,2018,AUDI,A3,NaN,NaN,NaN,0,0,0
2,King,Seattle,WA,2015,NISSAN,LEAF,NaN,NaN,NaN,135,386,18
3,King,Kent,WA,2019,AUDI,E-TRON,NaN,NaN,NaN,6,10,0
4,Snohomish,Bothell,WA,2022,TESLA,MODEL X,NaN,NaN,NaN,6,20,0
5,Snohomish,Lynnwood,WA,2016,KIA,SOUL,NaN,NaN,NaN,5,5,2
6,Snohomish,Edmonds,WA,2019,NISSAN,LEAF,NaN,NaN,NaN,5,10,4
7,Snohomish,Brier,WA,2020,KIA,NIRO,NaN,NaN,NaN,0,0,0
8,Kitsap,Bremerton,WA,2025,HYUNDAI,TUCSON,NaN,NaN,NaN,15,30,2
9,Sauk,Spring Green,WI,2021,JEEP,WRANGLER,NaN,NaN,NaN,0,0,0


## Step 7: Impute Remaining Missing Values (Specs Columns)

To ensure the final merged dataset is complete and ready for visualisation, I applied numerical and categorical imputation on all specification-related columns. Different strategies were used depending on the data type.

In [ ]:

# IMPUTATION FOR SPECS COLUMNS

import numpy as np

# NUMERIC COLUMNS - choose median or mean
num_median_cols = [
    'range_km', 'fast_charging_power_kw_dc', 'towing_capacity_kg',
    'number_of_cells', 'acceleration_0_100_s'
]

num_mean_cols = [
    'top_speed_kmh', 'battery_capacity_kWh', 'torque_nm',
    'efficiency_wh_per_km', 'length_mm', 'width_mm', 'height_mm'
]

# CATEGORICAL COLUMNS - mode
cat_cols = [
    'battery_type', 'fast_charge_port', 'cargo_volume_l',
    'drivetrain', 'segment', 'car_body_type'
]

# Apply median
for col in num_median_cols:
    if col in final_merged.columns:
        final_merged[col] = final_merged[col].fillna(final_merged[col].median())

# Apply mean
for col in num_mean_cols:
    if col in final_merged.columns:
        final_merged[col] = final_merged[col].fillna(final_merged[col].mean())

# Apply mode
for col in cat_cols:
    if col in final_merged.columns:
        final_merged[col] = final_merged[col].fillna(final_merged[col].mode()[0])

# Fill missing model names (very rare)
if 'model' in final_merged.columns:
    final_merged['model'] = final_merged['model'].fillna(final_merged['model'].mode()[0])


## Step 8: Replace Non-informative Strings with NaN

Some rows contained empty strings or text placeholders like "Not available" or "N/A".
These were replaced with proper NaN values so they could be imputed correctly.

In [ ]:
final_merged = final_merged.replace(["", " ", "Not available", "not available", "N/A"], np.nan)

## Step 9: Re-run Imputation After Cleaning Strings

After normalising text placeholders, median/mean/mode imputation was applied again to guarantee full completeness across all columns.

In [ ]:
for col in num_median_cols:
    if col in final_merged.columns:
        final_merged[col] = final_merged[col].fillna(final_merged[col].median())

for col in num_mean_cols:
    if col in final_merged.columns:
        final_merged[col] = final_merged[col].fillna(final_merged[col].mean())

for col in cat_cols:
    if col in final_merged.columns:
        final_merged[col] = final_merged[col].fillna(final_merged[col].mode()[0])

## Step 10: Remove Unnecessary or Duplicated Columns



In [ ]:
# Remove unused or poorly populated columns
final_merged = final_merged.drop(columns=['brand', 'seats', 'source_url'])

# Check missing values again
final_merged.isnull().sum()

,0
County,0
City,0
State,0
Postal Code,0
Model Year,0
Make,0
Model,0
Electric Vehicle Type,0
Clean Alternative Fuel Vehicle (CAFV) Eligibility,0
Electric Range,0


## Step 11: Remove Duplicate ‘model’ Column from Specs



In [ ]:
# Drop duplicate 'model' column from specs (keep EV 'Model')
if 'model' in final_merged.columns:
    final_merged = final_merged.drop(columns=['model'])

print(final_merged.columns)
print(final_merged.shape)


Index(['County', 'City', 'State', 'Postal Code', 'Model Year', 'Make', 'Model',
       'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Vehicle Location', 'Electric Utility', 'top_speed_kmh',
       'battery_capacity_kWh', 'battery_type', 'number_of_cells', 'torque_nm',
       'efficiency_wh_per_km', 'range_km', 'acceleration_0_100_s',
       'fast_charging_power_kw_dc', 'fast_charge_port', 'towing_capacity_kg',
       'cargo_volume_l', 'drivetrain', 'segment', 'length_mm', 'width_mm',
       'height_mm', 'car_body_type', 'num_stations', 'total_level1_ports',
       'total_level2_ports', 'total_dc_fast_ports'],
      dtype='object')
(264628, 35)


## Step 12: Save the Final Version of the Clean Merged Dataset


In [ ]:
final_save_path = '/content/drive/MyDrive/Projects - HA/DMV/final_merged_ev_dataset_cleaned.csv'
final_merged.to_csv(final_save_path, index=False)

print("Final cleaned merged dataset saved at:", final_save_path)
print("Shape:", final_merged.shape)

Final cleaned merged dataset saved at: /content/drive/MyDrive/Projects - HA/DMV/final_merged_ev_dataset_cleaned.csv
Shape: (264628, 35)


## Final Explanatory Visualisation: Animated Bubble Scatterplot

This final visualisation is designed to explain how electric vehicle adoption, vehicle capability, and charging infrastructure intensity evolve together over time across cities in Washington State. An animated bubble scatterplot was selected because it is particularly effective for representing multivariate, time-dependent quantitative relationships in a single coherent visual form.

This chart type allows the simultaneous comparison of infrastructure availability, vehicle performance, and adoption scale, while the animated time dimension reveals how these relationships change across successive model years. In the context of electric vehicles, where infrastructure growth, technological improvement, and adoption diffusion interact dynamically, a static chart would fail to capture these evolving dependencies. The animated bubble scatterplot therefore provides a rigorous and intuitive way to communicate both cross-sectional variation between cities and longitudinal change over time.

Prior to visualisation, the fully integrated dataset was structured at the city–model year level, enabling consistent temporal comparison while preserving meaningful geographic variation. This ensures that the animation reflects genuine system-wide trends rather than isolated fluctuations.


In [ ]:

# Animated bubble scatter over model year

df = final_merged.copy()

# Round model year to nearest integer for cleaner frames
df["ModelYear_int"] = df["Model Year"].round().astype(int)

# Re-aggregate per city + year
city_year = (
    df.groupby(["City", "State", "ModelYear_int"], as_index=False)
      .agg(
          n_vehicles=("Model", "count"),
          avg_range_km=("range_km", "mean"),
          num_stations=("num_stations", "mean"),
          total_level1_ports=("total_level1_ports", "mean"),
          total_level2_ports=("total_level2_ports", "mean"),
          total_dc_fast_ports=("total_dc_fast_ports", "mean")
      )
)

city_year["total_ports"] = (
    city_year["total_level1_ports"]
    + city_year["total_level2_ports"]
    + city_year["total_dc_fast_ports"]
)

city_year["ports_per_100_evs"] = (
    city_year["total_ports"] / city_year["n_vehicles"] * 100
)

# Filter
city_year_filtered = city_year[city_year["n_vehicles"] >= 30].copy()

fig_anim = px.scatter(
    city_year_filtered,
    x="ports_per_100_evs",
    y="avg_range_km",
    size="n_vehicles",
    color="ports_per_100_evs",
    animation_frame="ModelYear_int",
    hover_name="City",
    hover_data={
        "State": True,
        "n_vehicles": True,
        "avg_range_km": ":.1f",
        "ports_per_100_evs": ":.1f",
        "num_stations": True,
        "total_ports": True
    },
    labels={
        "ports_per_100_evs": "Charging ports per 100 EVs",
        "avg_range_km": "Average EV range (km)",
        "n_vehicles": "Number of EV registrations",
        "ModelYear_int": "Model year"
    },
    title="How the EV–Charging Relationship Changes Across Model Years"
)

fig_anim.update_layout(template="plotly_white")
fig_anim.show()


## How the Final Visualisation Works

The final visualisation is implemented as an interactive animated bubble scatterplot using Plotly. Each animation frame represents a single model year, allowing users to observe how the relationship between electric vehicle adoption, charging infrastructure, and vehicle capability changes over time. Cities are used as the unit of comparison and move dynamically as their infrastructure intensity and average vehicle range evolve.

The design combines multiple dimensions into a single view, enabling direct comparison of adoption scale, infrastructure availability, and vehicle performance across cities and across time. To ensure clarity and stability, cities with very small EV counts were filtered out prior to plotting.

Key features of the visualisation include:
- **Bubble size** represents the total number of registered EVs in each city, indicating adoption scale.  
- **Animation over model years** allows users to track long-term trends in infrastructure growth and vehicle capability.


## Interpretation and Conclusion of the Final Visualisation

The animated bubble scatterplot reveals a clear and consistent relationship between charging infrastructure intensity, electric vehicle adoption, and vehicle capability across Washington State. Over successive model years, most cities show a general movement towards higher charging availability and higher average vehicle range, indicating parallel technological and infrastructure growth alongside rising EV adoption.

The visualisation also highlights important imbalances in certain locations:
- Some cities show **rapid EV adoption without matching infrastructure growth**.
- These appear as **large bubbles in low infrastructure regions**, signalling potential future pressure on public charging networks.

Overall, the animated visualisation transforms a complex, multi-source dataset into a clear temporal narrative that is both interpretable and policy-relevant. It confirms the critical role of charging infrastructure in enabling large-scale EV adoption while also identifying cities where infrastructure investment may need to be prioritised to support sustainable transport growth.